<a href="https://colab.research.google.com/github/zeiosis/ffnet-summary-prediction/blob/main/dangerzone/colab/ff_analysis_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pth_tags = 'gen_ffa'
pth_comm_msg = '1st training'
basis_checkpoint = 'zdreiosis/ff_analysis_5'
csvname = 'ff_dump_final2.csv'
save_checkpoint_name = f"ff_analysis_5" #formatted like this in the original -- indicates name of model to upload to
epoch_quant = 3

#Libraries and definitions

In [ ]:
!pip install -U transformers==4.15.0 --quiet
!pip install -U sentence-transformers==2.2.0 --quiet
!pip install -U transformers[sentencepiece] --quiet
!pip install datasets --quiet

import transformers
from transformers import pipeline, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, AutoModelForSequenceClassification, EvalPrediction
import pandas as pd
from datasets import Dataset, DatasetDict
import numpy as np
from huggingface_hub import notebook_login
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import torch

     |████████████████████████████████| 3.4 MB 23.5 MB/s 
     |████████████████████████████████| 3.3 MB 62.7 MB/s 
     |████████████████████████████████| 880 kB 25.5 MB/s 
     |████████████████████████████████| 596 kB 43.9 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 79 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 38.9 MB/s 
     |████████████████████████████████| 362 kB 33.1 MB/s 
     |████████████████████████████████| 140 kB 73.0 MB/s 
     |████████████████████████████████| 1.1 MB 61.0 MB/s 
     |████████████████████████████████| 212 kB 69.7 MB/s 
     |████████████████████████████████| 127 kB 73.5 MB/s 
     |████████████████████████████████| 144 kB 74.3 MB/s 
     |████████████████████████████████| 271 kB 62.8 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [ ]:
def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label] #definitely not my code sadface

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

#CUDA

In [ ]:
print(f"Is CUDA supported by this system? \
      {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:\
      {torch.cuda.current_device()}")
        
print(f"Name of current CUDA device:\
      {torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system?       True
CUDA version: 11.3
ID of current CUDA device:      0
Name of current CUDA device:      Tesla T4


#Data processing

Checkpoint

In [ ]:
checkpoint = basis_checkpoint 

Data adaptation

In [ ]:
raw_datasets = pd.read_csv(csvname) #training data, in df-to-csv'ed form. original df should be one-hot.
raw_datasets = raw_datasets.rename(columns={'Summary': 'text', 'Unnamed: 0': 'ID'}) #i'm too lazy to figure out how to change the code

raw_datasets = Dataset.from_dict(raw_datasets)

# 90% train, 10% test + validation
ttv_ds = raw_datasets.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = ttv_ds['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
raw_datasets = DatasetDict({
    'train': ttv_ds['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) 

labels = [label for label in raw_datasets['train'].features.keys() if label not in ['ID', 'text']] #finds labels ('genres') in columns
id2label = {idx:label for idx, label in enumerate(labels)} 
label2id = {label:idx for idx, label in enumerate(labels)} #creates dicts for id/label (&v/v) mappings

encoded_dataset = raw_datasets.map(preprocess_data, batched=True, remove_columns=raw_datasets['train'].column_names)
encoded_dataset.set_format("torch")

Downloading:   0%|          | 0.00/327 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Parameter 'function'=<function preprocess_data at 0x7f5b18e5cc20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

#[Manual]: HF login

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


#Model setup

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(basis_checkpoint,  ###goddammit not again
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
batch_size = 32
metric_name = "f1"

In [ ]:
args = TrainingArguments(
    save_checkpoint_name,
    evaluation_strategy = "steps", #epoch
    #save_strategy = "no", #epoch
    eval_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch_quant,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

In [ ]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result #pretty sure I have to change this somehow to make a custom loss fx? otherwise reported will be misleading, but does this impact the training?

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))

#Training

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/zdreiosis/ff_analysis_5 into local empty directory.


Download file pytorch_model.bin:   0%|          | 3.47k/418M [00:00<?, ?B/s]

Download file runs/Jun18_14-09-51_68a0a0482637/events.out.tfevents.1655561554.68a0a0482637.74.0:  42%|####2   …

Download file runs/Jun18_12-49-56_512e75246bcc/events.out.tfevents.1655556618.512e75246bcc.67.0:  42%|####2   …

Download file runs/Jun18_13-44-48_135238e3f814/events.out.tfevents.1655560042.135238e3f814.72.0:  42%|####2   …

...

Download file runs/Jun18_12-49-56_512e75246bcc/events.out.tfevents.1655557033.512e75246bcc.67.2: 100%|########…

Clean file runs/Jun18_13-13-10_5a3f2dde8da2/events.out.tfevents.1655558587.5a3f2dde8da2.72.2: 100%|##########|…

Clean file runs/Jun18_14-09-51_68a0a0482637/events.out.tfevents.1655561996.68a0a0482637.74.2: 100%|##########|…

Clean file runs/Jun18_12-49-56_512e75246bcc/events.out.tfevents.1655557033.512e75246bcc.67.2: 100%|##########|…

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 5798
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 546


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
50,No log,0.084649,0.930502,0.947626,0.807453
100,No log,0.100003,0.906977,0.932022,0.748447
150,No log,0.094493,0.912621,0.934880,0.763975
200,No log,0.097342,0.911907,0.935331,0.776398
250,No log,0.087958,0.933590,0.950445,0.826087
300,No log,0.085703,0.924565,0.943377,0.804348
350,No log,0.084419,0.932432,0.948810,0.819876
400,No log,0.088064,0.923225,0.944974,0.788820
450,No log,0.087482,0.927676,0.946197,0.801242
500,0.122600,0.082440,0.930636,0.948321,0.813665


***** Running Evaluation *****
  Num examples = 322
  Batch size = 32
***** Running Evaluation *****
  Num examples = 322
  Batch size = 32
***** Running Evaluation *****
  Num examples = 322
  Batch size = 32
***** Running Evaluation *****
  Num examples = 322
  Batch size = 32
***** Running Evaluation *****
  Num examples = 322
  Batch size = 32
***** Running Evaluation *****
  Num examples = 322
  Batch size = 32
***** Running Evaluation *****
  Num examples = 322
  Batch size = 32
***** Running Evaluation *****
  Num examples = 322
  Batch size = 32
***** Running Evaluation *****
  Num examples = 322
  Batch size = 32
***** Running Evaluation *****
  Num examples = 322
  Batch size = 32
Saving model checkpoint to ff_analysis_5/checkpoint-500
Configuration saved in ff_analysis_5/checkpoint-500/config.json
Model weights saved in ff_analysis_5/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ff_analysis_5/checkpoint-500/tokenizer_config.json
Special tokens file saved in

TrainOutput(global_step=546, training_loss=0.1188740603653066, metrics={'train_runtime': 423.0664, 'train_samples_per_second': 41.114, 'train_steps_per_second': 1.291, 'total_flos': 1144200060776448.0, 'train_loss': 0.1188740603653066, 'epoch': 3.0})

#Results

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 322
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.8136645962732919,
 'eval_f1': 0.930635838150289,
 'eval_loss': 0.08243951201438904,
 'eval_roc_auc': 0.9483211453290762,
 'eval_runtime': 2.5612,
 'eval_samples_per_second': 125.72,
 'eval_steps_per_second': 4.295}

In [ ]:
trainer.push_to_hub(tags=pth_tags, commit_message=pth_comm_msg) 

Saving model checkpoint to ff_analysis_5
Configuration saved in ff_analysis_5/config.json
Model weights saved in ff_analysis_5/pytorch_model.bin
tokenizer config file saved in ff_analysis_5/tokenizer_config.json
Special tokens file saved in ff_analysis_5/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/418M [00:00<?, ?B/s]

Upload file runs/Jun18_14-33-45_348f48bfeaf4/events.out.tfevents.1655563015.348f48bfeaf4.75.0:  41%|####      …

Upload file runs/Jun18_14-33-45_348f48bfeaf4/events.out.tfevents.1655563441.348f48bfeaf4.75.2: 100%|##########…

To https://huggingface.co/zdreiosis/ff_analysis_5
   54cffbc..1c7d5f4  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 0.930635838150289}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.8136645962732919}]}
To https://huggingface.co/zdreiosis/ff_analysis_5
   1c7d5f4..81b3032  main -> main



'https://huggingface.co/zdreiosis/ff_analysis_5/commit/1c7d5f48c83e5222c2a8dbd68654ed5744a2ec2c'